In [3]:
from datetime import datetime
import pickle

In [4]:
class file_system:
    
    def __init__(self):
        
        self.size_A = 200
        self.size_B = 300
        self.total_size = self.size_A + self.size_B
        self.disk_A_data = ['0'*100]*self.size_A
        self.disk_A_metadata = [0]*self.size_A
        self.disk_B_data = ['0'*100]*self.size_B
        self.disk_B_metadata = [0]*self.size_B
    
    def iscorrect(self,block_number):
        if block_number>0 and block_number<=500:
            return True
        return False
    
    def resolve_block_number(self,block_number):
        if block_number<=200:
            return 'A',block_number-1
        else:
            return 'B',block_number-201
    
    def read_block_data(self,block_number):
        disk, block_number = self.resolve_block_number(block_number)
        if disk=='A':
            return self.disk_A_data[block_number][:self.disk_A_metadata[block_number]]
        elif disk=='B':
            return self.disk_B_data[block_number][:self.disk_B_metadata[block_number]]
        
    def write_block_data(self,block_number,block_info):
        disk, block_number = self.resolve_block_number(block_number)
        data_length = len(block_info)
        block_info += '0'*100
        block_info = block_info[:100]
        if disk=='A':
            self.disk_A_metadata[block_number] = data_length
            self.disk_A_data[block_number] = block_info
        elif disk=='B':
            self.disk_B_metadata[block_number] = data_length
            self.disk_B_data[block_number] = block_info

In [26]:
class suraj(file_system):
    
    def __init__(self):
        super().__init__()
        self.disk_dictionary = dict()
        self.space_list = [[0,self.total_size-1,0]]
        self.space_available = self.total_size
        self.next_available_id = 11
    
    def create_disk(self,disk_size):
        if self.space_available < disk_size:
            return -1
        disk_id = 'virtual_disk_'+str(self.next_available_id)
        self.next_available_id += 1
        self.space_available -= disk_size
        self.disk_dictionary[disk_id] = []
        for i in range(len(self.space_list)):
            if not self.space_list[i][2]:
                if (self.space_list[i][1]-self.space_list[i][0]+1) < disk_size:
                    self.space_list[i][2] = 1
                    self.disk_dictionary[disk_id] += [self.space_list[i][:2]]
                    disk_size -= (self.space_list[i][1]-self.space_list[i][0]+1)
                elif (self.space_list[i][1]-self.space_list[i][0]+1) == disk_size:
                    self.space_list[i][2] = 1
                    self.disk_dictionary[disk_id] += [self.space_list[i][:2]]
                    break
                elif (self.space_list[i][1]-self.space_list[i][0]+1) > disk_size:
                    temp = self.space_list[i][1]
                    self.space_list[i][1] = self.space_list[i][0] + disk_size - 1
                    self.space_list[i][2] = 1
                    self.space_list.insert(i+1,[self.space_list[i][1]+1,temp,0])
                    self.disk_dictionary[disk_id] += [self.space_list[i][:2]]
                    break
        return disk_id
        
    def display_virtual_disks_names(self):
        print('List of all Virtual Disks :')
        for (i,j) in enumerate(list(self.disk_dictionary.keys())):
            print(str(i+1)+'. '+str(j))
            
    def resolve_disk_number(self,disk_number):
        if disk_number<=0 or disk_number>len(self.disk_dictionary):
            return -1
        return list(self.disk_dictionary.keys())[disk_number-1]
    
    def delete_disk(self,disk_id):
        
        space = 0
        for i in self.disk_dictionary[disk_id]:
            space += i[1]-i[0]+1
        self.space_available += space
        
        for i in self.disk_dictionary[disk_id]:
            for j in range(len(self.space_list)):
                if self.space_list[j][0]==i[0] and self.space_list[j][1]==i[1]:
                    self.space_list[j][2] = 0
        temp = [self.space_list[0]]
        self.space_list = self.space_list[1:]
        while self.space_list:
            if temp[-1][2]==0 and self.space_list[0][2]==0:
                temp[-1][1] = self.space_list[0][2]
            else:
                temp.append(self.space_list[0])
            self.space_list = self.space_list[1:]
        self.space_list = temp
        
        del self.disk_dictionary[disk_id]
        
        
    def resolve_virtual_disk_block_number(self,disk_number,block_number):
        
        disk_id = self.resolve_disk_number(disk_number)
        if disk_id==-1:
            return -1
        disk_size = 0        
        for i in self.disk_dictionary[disk_id]:
            disk_size += i[1]-i[0]+1
        if block_number<=0 or block_number>disk_size:
            return -1
        
        print(disk_id)
        
        block_number -= 1
        for i in self.disk_dictionary[disk_id]:
            if i[1]-i[0] < block_number:
                block_number -= i[1]-i[0]+1
            else:
                block_number += i[0]
                return block_number
        
    def read_virtual_disk_data(self,disk_number,block_number):
        
        block_number = self.resolve_virtual_disk_block_number(disk_number,block_number)
        if block_number == -1:
            return -1
        print(block_number)
        return self.read_block_data(block_number)
    
    def write_virtual_disk_data(self,disk_number,block_number,block_information):
        
        block_number = self.resolve_virtual_disk_block_number(disk_number,block_number)
        if block_number == -1:
            return -1
        return self.write_block_data(block_number,block_information)

In [ ]:
class question1:
    def main(self):
        while(True):
            print('******************************************')
            print('Choose:')
            print('1. Read data from disk.')
            print('2. Write data to disk.')
            print('3. Display all the current virtual disks.')
            print('4. Create a virtual disk.')
            print('5. Delete a virtual disk.')
            print('9. Exit')
            print('******************************************')
            option = int(input('Enter your option : '))
            if option==1:
                block_number = int(input('Enter a block number between 1-500 : '))
                print('\n'+self.read_block_data(block_number)+'\n')
            elif option==2:
                block_number = int(input('Enter a block number between 1-500 : '))
                block_info = input('Enter the information to be entered into the block : ')
                self.write_block_data(block_number,block_info)
                print('\nData written to the desired block.\n')
            elif option==3:
                print('\nBackup name: '+self.create_backup()+'. Created Successfully.\n')
            elif option==4:
                backup_name = self.restore_backup()
                print('\nRestored to:'+backup_name+'.\n')
            elif option==9:
                break
            else:
                print('Please enter a valid option.')

In [34]:
system = suraj()

system.create_disk(100)
system.create_disk(10)
system.create_disk(320)
system.create_disk(10)
system.create_disk(20)

system.delete_disk('virtual_disk_12')
system.delete_disk('virtual_disk_14')

print(system.disk_dictionary)
print(system.space_list)
print(system.space_available)
system.display_virtual_disks_names()

system.write_virtual_disk_data(3,1,'konark')
print(system.read_virtual_disk_data(3,1))
print(system.read_block_data(440))

{'virtual_disk_11': [[0, 99]], 'virtual_disk_13': [[110, 429]], 'virtual_disk_15': [[440, 459]]}
[[0, 99, 1], [100, 109, 0], [110, 429, 1], [430, 439, 0], [440, 459, 1], [460, 499, 0]]
60
List of all Virtual Disks :
1. virtual_disk_11
2. virtual_disk_13
3. virtual_disk_15
virtual_disk_15
virtual_disk_15
440
konark
konark


In [15]:
class snapshot_System(virtual_System):
    
    def __init__(self):
        super().__init__()
        self.backup_metadata = []
        
    def get_backup_name(self):
        print('Choose the version that you wish to restore your system to:')
        for (backup_number,backup_name) in enumerate(self.backup_metadata):
            print(backup_number+1,backup_name)
        backup_number = int(input())
        if backup_number>0 and backup_number<=len(self.backup_metadata):
            return self.backup_metadata[backup_number-1]
        else:
            print('Please enter a correct number:')
            return self.get_backup_name()
        
    def create_backup(self):
        backup_name = str(datetime.now())[:-4].replace(':','-').replace('.','-')
        self.backup_metadata.append(backup_name)
        file = open(backup_name,'wb')
        pickle.dump([self.disk_A_data,self.disk_A_metadata,self.disk_B_data,self.disk_B_metadata],file)
        file.close()
        return backup_name
        
    def restore_backup(self):
        backup_name = self.get_backup_name()
        file = open(backup_name,'rb')
        [self.disk_A_data,self.disk_A_metadata,self.disk_B_data,self.disk_B_metadata] = pickle.load(file)
        file.close()
        return backup_name
        
    def main(self):
        while(True):
            print('******************************************')
            print('Choose:')
            print('1. Read data from disk.')
            print('2. Write data to disk.')
            print('3. Create Backup of the disk.')
            print('4. Restore to a particular backup.')
            print('5. Exit')
            print('******************************************')
            option = int(input('Enter your option : '))
            if option==1:
                block_number = int(input('Enter a block number between 1-500 : '))
                print('\n'+self.read_block_data(block_number)+'\n')
            elif option==2:
                block_number = int(input('Enter a block number between 1-500 : '))
                block_info = input('Enter the information to be entered into the block : ')
                self.write_block_data(block_number,block_info)
                print('\nData written to the desired block.\n')
            elif option==3:
                print('\nBackup name: '+self.create_backup()+'. Created Successfully.\n')
            elif option==4:
                backup_name = self.restore_backup()
                print('\nRestored to:'+backup_name+'.\n')
            elif option==5:
                break
            else:
                print('Please enter a valid option.')

In [18]:
snapshot_System().main()

******************************************
Choose:
1. Read data from disk.
2. Write data to disk.
3. Create Backup of the disk.
4. Restore to a particular backup.
5. Exit
******************************************
Enter your option : 2
Enter a block number between 1-500 : 12
Enter the information to be entered into the block : koanrk

Data written to the desired block.

******************************************
Choose:
1. Read data from disk.
2. Write data to disk.
3. Create Backup of the disk.
4. Restore to a particular backup.
5. Exit
******************************************
Enter your option : 2
Enter a block number between 1-500 : 34
Enter the information to be entered into the block : sriraam

Data written to the desired block.

******************************************
Choose:
1. Read data from disk.
2. Write data to disk.
3. Create Backup of the disk.
4. Restore to a particular backup.
5. Exit
******************************************
Enter your option : 3

Backup name: 201

ValueError: invalid literal for int() with base 10: ''

In [9]:
mysystem = snapshot_System()
mysystem.create_backup()
mysystem.create_backup()
mysystem.create_backup()
mysystem.restore_backup()
mysystem.restore_backup()


Choose the version that you wish to restore your system to:
1 2019-09-14 10-09-17-03
2 2019-09-14 10-09-17-03
3 2019-09-14 10-09-17-04
1
Choose the version that you wish to restore your system to:
1 2019-09-14 10-09-17-03
2 2019-09-14 10-09-17-03
3 2019-09-14 10-09-17-04
2
